In [1]:
lb  = 50
print(ukbb_thr_stats[ukbb_thr_stats.mea_b < lb].shape)

for i in ukbb_thr_stats[ukbb_thr_stats.mea_b < lb].file.sample(n=20):
    x = Fundus(f"{data_path}/{i}")
    x.show()

NameError: name 'ukbb_thr_stats' is not defined

In [ ]:
ub = 70
print(ukbb_thr_stats[ukbb_thr_stats.mea_r > ub].shape)

for i in ukbb_thr_stats[ukbb_thr_stats.mea_r > ub].file.sample(n=20):
    x = Fundus(f"{data_path}/{i}")
    x.show()